# 01 — Data Loading & Scope
Notebook ini berisi proses pengolahan data emisi karbon.
Langkah yang dilakukan adalah:
* Mengambil data emisi karbon dari sumber data (format CSV)
* Melihat gambaran umum data (jumlah baris/kolom, contoh isi data, dan tipe data)
* Setelah data dipahami, saya melakukan filter data menjadi negara ASEAN dan rentang tahun 2000–2024

Tujuan dari bagian ini adalah memastikan dataset sudah berhasil dipanggil dengan benar sebelum masuk ke tahap analisis lebih lanjut.

In [1]:
import pandas as pd
import numpy as np

## Data Collection
Dataset emisi karbon diambil dari file CSV yang disimpan di GitHub.
Setelah data dipanggil menjadi dataframe, beberapa baris awal ditampilkan untuk memastikan data terbaca dengan benar.


In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv", delimiter=',')
df.head()


,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1750,AFG,2802560.0,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1751,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1752,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1753,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1754,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Dataset ini berisi informasi emisi untuk banyak negara dengan rentang tahun 1750–2024.

Sebelum melakukan filter, dilakukan pengecekan jumlah baris dan kolom.

In [3]:
df.shape

(50411, 79)

Dataset memiliki 50.195 baris dan 79 kolom.
Jumlah ini mencakup seluruh negara dan rentang waktu panjang.

Untuk memahami struktur data, dicek daftar kolom yang tersedia.

In [4]:
df.columns


Index(['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct',
       'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_chang

Kolom yang tersedia mencakup informasi negara, tahun, serta berbagai metrik emisi karbon dan gas rumah kaca lainnya.

Selanjutnya dicek informasi tipe data dan missing value secara ringkas.

In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50411 entries, 0 to 50410
Data columns (total 79 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   country                                    50411 non-null  object 
 1   year                                       50411 non-null  int64  
 2   iso_code                                   42480 non-null  object 
 3   population                                 41167 non-null  float64
 4   gdp                                        15251 non-null  float64
 5   cement_co2                                 29173 non-null  float64
 6   cement_co2_per_capita                      25648 non-null  float64
 7   co2                                        29384 non-null  float64
 8   co2_growth_abs                             27216 non-null  float64
 9   co2_growth_prct                            26239 non-null  float64
 10  co2_including_luc     

Terdapat kombinasi kolom numerik dan kategorikal.
Beberapa kolom memiliki missing value, yang akan dievaluasi lebih lanjut pada tahap data quality check

## Define Categorical Columns (Deterministik)
Kolom kategorikal diidentifikasi dan diurutkan secara deterministik agar hasil konsisten saat notebook dijalankan ulang.

In [6]:
cat_cols = sorted(df.select_dtypes(include="object").columns.tolist())
cat_cols


['country', 'iso_code']

### Filter Negara ASEAN
Scope analisis dibatasi pada negara anggota ASEAN.

In [7]:
asean_countries = [
    "Brunei",
    "Cambodia",
    "Indonesia",
    "Laos",
    "Malaysia",
    "Myanmar",
    "Philippines",
    "Singapore",
    "Thailand",
    "Vietnam"
]


Dataset difilter agar hanya memuat negara ASEAN.

### Filter Tahun 2000–2024
Periode analisis dibatasi pada tahun 2000 hingga 2024.

In [8]:
df = df[(df["year"] >= 2000) & (df["year"] <= 2024)].copy()
df.shape


(6350, 79)

In [9]:
df.head()


,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
250,Afghanistan,2000,AFG,20130334.0,1.128379e+10,0.010,0.001,1.047,-0.045,-4.078,...,NaN,0.078,0.0,0.001,0.001,0.0,16.009,2.603,NaN,NaN
251,Afghanistan,2001,AFG,20284303.0,1.102127e+10,0.007,0.000,1.069,0.022,2.098,...,NaN,0.077,0.0,0.001,0.001,0.0,14.895,2.701,NaN,NaN
252,Afghanistan,2002,AFG,21378123.0,1.880487e+10,0.011,0.001,1.341,0.272,25.439,...,NaN,0.077,0.0,0.001,0.001,0.0,20.037,3.044,NaN,NaN
253,Afghanistan,2003,AFG,22733053.0,2.107434e+10,0.010,0.000,1.560,0.219,16.302,...,NaN,0.076,0.0,0.001,0.001,0.0,20.769,3.349,NaN,NaN
254,Afghanistan,2004,AFG,23560656.0,2.233257e+10,0.010,0.000,1.237,-0.322,-20.673,...,NaN,0.076,0.0,0.001,0.001,0.0,18.934,3.085,NaN,NaN


Contoh baris data ditampilkan untuk memastikan hasil filter sesuai.

### Validasi Dasar dan Simpan Output Processed
Validasi dasar dilakukan untuk memastikan konsistensi data setelah filter.
Output processed disimpan agar notebook lanjutan menggunakan sumber data yang sama

In [10]:
from pathlib import Path

print("Countries:", sorted(df["country"].unique()))
print("Year min/max:", int(df["year"].min()), int(df["year"].max()))
print("Rows:", len(df), "Cols:", df.shape[1])

Countries: ['Afghanistan', 'Africa', 'Africa (GCP)', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Asia', 'Asia (GCP)', 'Asia (excl. China and India)', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Central African Republic', 'Central America (GCP)', 'Chad', 'Chile', 'China', 'Christmas Island', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Curacao', 'Cyprus', 'Czechia', 'Democratic Republic of Congo', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'East Timor', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Est

menampilkan daftar negara unik yang diurutkan, rentang tahun minimum–maksimum, serta jumlah baris dan kolom pada df.

In [11]:
dup_country_year = df.duplicated(["country", "year"]).sum()
print("Duplicate country-year:", int(dup_country_year))

Duplicate country-year: 0


menghitung jumlah baris dengan pasangan country–year yang duplikat lalu mencetak total duplikat tersebut sebagai bilangan bulat.

In [12]:
coverage = (
    df.groupby("country")["year"]
    .agg(["min", "max", "nunique"])
    .sort_values("nunique")
)
coverage

,min,max,nunique
country,,,
Afghanistan,2000,2024,25
Africa,2000,2024,25
Africa (GCP),2000,2024,25
Albania,2000,2024,25
Algeria,2000,2024,25
...,...,...,...
Wallis and Futuna,2000,2024,25
World,2000,2024,25
Yemen,2000,2024,25


Membuat ringkasan per negara berisi tahun minimum, maksimum, dan jumlah tahun unik lalu mengurutkannya berdasarkan nunique (jumlah tahun unik) dari yang paling sedikit.

In [13]:
out_dir = Path("data/process")
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "owid_co2_asean_2000_2024.csv"
df.to_csv(out_path, index=False)

out_path.resolve()


PosixPath('/content/data/process/owid_co2_asean_2000_2024.csv')

Output menunjukkan dataset telah terfokus pada negara ASEAN dengan periode 2000–2024.
Dataset ini menjadi input utama untuk tahap data quality check dan EDA pada notebook berikutnya.

**Conclusion**  
Data emisi karbon berhasil dimuat dari OWID, lalu difilter pada negara ASEAN dan periode 2000–2024, divalidasi secara dasar (rentang tahun, duplikasi country–year, dan cakupan per negara), serta disimpan sebagai dataset terproses untuk digunakan pada tahap data quality check dan EDA berikutnya.